# Evaluation (TODO Latex figures without Type3 fonts!!!)

- Tables: Overall / By Label Class
- Figures: By Citation Count / By Year (difference)
- Samples (ACL only?)
- Confusion matrix?

```bash
# Compress all result files (exclude model weights)
tar -cvzf acl_docrel.tar.gz --exclude='*.bin' acl_docrel/*
```


In [439]:
import re
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from collections import defaultdict
from experiments.utils import highlight_max
import matplotlib.patches as mpatches
from matplotlib.container import ErrorbarContainer

import os
import random
import json
import pickle
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support
from IPython.core.display import display

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

INFO:transformers.file_utils:PyTorch version 1.4.0 available.
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [180]:
output_dir = './output/'
folds = 4

metric_cols = {
    'micro avg__precision': 'Precision (micro)',
    'micro avg__recall': 'Recall (micro)',
    'micro avg__f1-score': 'F1 (micro)',
    'macro avg__precision': 'Precision (macro)',
    'macro avg__recall': 'Recall (macro)',
    'macro avg__f1-score': 'F1 (macro)',
}

# Overall results

Table: System, P, R, F micro (std) F macro (std) | for ACL-Anthology + CORD-19


In [359]:
overall_rows = []
exp_names = []

for exp in os.listdir(output_dir):
    exp_dir = os.path.join(output_dir, exp)

    if not exp.endswith('_docrel') or os.path.isfile(exp_dir):
        continue
    
    print(f'Experiment: {exp}')
    
    for fold in range(1, folds+1):
        fold_dir = os.path.join(exp_dir, 'folds', str(fold))
        print(f'- Fold: {fold}')

        if not os.path.exists(fold_dir):
            logger.warning(f'Fold does not exist: {fold_dir}')
            continue

        for sys in os.listdir(fold_dir):
            sys_dir = os.path.join(fold_dir, sys)

            if not os.path.isdir(sys_dir) or sys.startswith('.') or sys.startswith('__'):
                continue

            try:
                metrics = json.load(open(os.path.join(sys_dir, 'metrics.json')))
                metrics = {k.replace('eval_', ''): v for k, v in metrics.items()}
                exp_name = exp.replace('acl_docrel', 'ACL Anthology').replace('cord19_docrel', 'CORD-19')
                row = {                    
                    'system': sys,
                    'experiment': exp_name,
                    'fold': fold,
                    'class': None,
                    'precision': 0.,
                    'recall': 0.,
                    'f1-score': 0.,
                    'support': 0,
                }
                cs = set([k.split('__')[0] for k in metrics.keys() if len(k.split('__')) == 2])

                for c in cs:
                    crow = row.copy()
                    crow['class'] = c
                    for m in ms:
                        crow[m] = metrics[c + '__' + m]
                    overall_rows.append(crow)
                    
                print(f'   - System: {sys}')

            except FileNotFoundError:
                logger.warning(f'Skip {sys_dir} (not all files exists, probably still running..)')
                pass
    #break

Experiment: acl_docrel
- Fold: 1
   - System: roberta-base
   - System: electra-base-discriminator
   - System: bert-base-cased
   - System: baseline-rnn__fasttext__custom
   - System: xlnet-base-cased
   - System: baseline-rnn
   - System: scibert-scivocab-uncased
   - System: covid_bert_base
- Fold: 2
   - System: roberta-base
   - System: electra-base-discriminator
   - System: bert-base-cased
   - System: baseline-rnn__fasttext__custom
   - System: xlnet-base-cased
   - System: baseline-rnn
   - System: scibert-scivocab-uncased
   - System: covid_bert_base
- Fold: 3
   - System: roberta-base
   - System: electra-base-discriminator
   - System: bert-base-cased
   - System: baseline-rnn__fasttext__custom
   - System: xlnet-base-cased
   - System: baseline-rnn
   - System: scibert-scivocab-uncased
   - System: covid_bert_base
- Fold: 4
   - System: roberta-base
   - System: electra-base-discriminator
   - System: bert-base-cased
   - System: baseline-rnn__fasttext__custom
   - System:

In [360]:
df = pd.DataFrame(overall_rows)

In [361]:
df

,system,experiment,fold,class,precision,recall,f1-score,support
0,roberta-base,ACL Anthology,1,evaluation,0.000000,0.000000,0.000000,253
1,roberta-base,ACL Anthology,1,discussion,0.000000,0.000000,0.000000,296
2,roberta-base,ACL Anthology,1,introduction,0.508932,0.354996,0.418250,4093
3,roberta-base,ACL Anthology,1,micro avg,0.707644,0.541440,0.613484,21646
4,roberta-base,ACL Anthology,1,none,0.905441,0.951392,0.927848,6069
...,...,...,...,...,...,...,...,...
507,covid_bert_base,ACL Anthology,4,conclusion,0.000000,0.000000,0.000000,281
508,covid_bert_base,ACL Anthology,4,related work,0.634146,0.556905,0.593021,3128
509,covid_bert_base,ACL Anthology,4,other,0.607843,0.589046,0.598297,5368
510,covid_bert_base,ACL Anthology,4,results,0.000000,0.000000,0.000000,286


In [362]:
systems = df['system'].unique().tolist()

In [363]:
for sys in sorted(systems):
    print(sys)
    display(df[(df['system'] == sys) & (df['class'] == 'micro avg')])
    #display(df[(df['system'] == sys) & (df['class'] == 'macro avg')])
    
    

baseline-rnn


,system,experiment,fold,class,precision,recall,f1-score,support
83,baseline-rnn,ACL Anthology,1,micro avg,0.760135,0.198374,0.314636,21646
211,baseline-rnn,ACL Anthology,2,micro avg,0.750000,0.202060,0.318351,21558
339,baseline-rnn,ACL Anthology,3,micro avg,0.791186,0.197907,0.316616,21500
467,baseline-rnn,ACL Anthology,4,micro avg,0.787380,0.203705,0.323671,21379


baseline-rnn__fasttext__custom


,system,experiment,fold,class,precision,recall,f1-score,support
51,baseline-rnn__fasttext__custom,ACL Anthology,1,micro avg,0.771262,0.176799,0.287658,21646
179,baseline-rnn__fasttext__custom,ACL Anthology,2,micro avg,0.752587,0.185546,0.297697,21558
307,baseline-rnn__fasttext__custom,ACL Anthology,3,micro avg,0.781591,0.183256,0.296899,21500
435,baseline-rnn__fasttext__custom,ACL Anthology,4,micro avg,0.759660,0.192198,0.306779,21379


bert-base-cased


...

,system,experiment,fold,class,precision,recall,f1-score,support
3,roberta-base,ACL Anthology,1,micro avg,0.707644,0.541440,0.613484,21646
131,roberta-base,ACL Anthology,2,micro avg,0.702251,0.561462,0.624014,21558
259,roberta-base,ACL Anthology,3,micro avg,0.705830,0.556326,0.622223,21500
387,roberta-base,ACL Anthology,4,micro avg,0.702513,0.561018,0.623843,21379


scibert-scivocab-uncased


,system,experiment,fold,class,precision,recall,f1-score,support
99,scibert-scivocab-uncased,ACL Anthology,1,micro avg,0.721928,0.617805,0.665820,21646
227,scibert-scivocab-uncased,ACL Anthology,2,micro avg,0.728194,0.620002,0.669757,21558
355,scibert-scivocab-uncased,ACL Anthology,3,micro avg,0.727209,0.628512,0.674268,21500
483,scibert-scivocab-uncased,ACL Anthology,4,micro avg,0.722643,0.620562,0.667724,21379


xlnet-base-cased


,system,experiment,fold,class,precision,recall,f1-score,support
67,xlnet-base-cased,ACL Anthology,1,micro avg,0.704042,0.584219,0.638558,21646
195,xlnet-base-cased,ACL Anthology,2,micro avg,0.702576,0.595974,0.644899,21558
323,xlnet-base-cased,ACL Anthology,3,micro avg,0.700539,0.598977,0.645789,21500
451,xlnet-base-cased,ACL Anthology,4,micro avg,0.700089,0.589176,0.639862,21379


In [364]:
df[(df['system'] == 'xlnet-base-cased') & (df['class'] == 'micro avg')]

,system,experiment,fold,class,precision,recall,f1-score,support
67,xlnet-base-cased,ACL Anthology,1,micro avg,0.704042,0.584219,0.638558,21646
195,xlnet-base-cased,ACL Anthology,2,micro avg,0.702576,0.595974,0.644899,21558
323,xlnet-base-cased,ACL Anthology,3,micro avg,0.700539,0.598977,0.645789,21500
451,xlnet-base-cased,ACL Anthology,4,micro avg,0.700089,0.589176,0.639862,21379


In [365]:
df.columns

Index(['system', 'experiment', 'fold', 'class', 'precision', 'recall',
       'f1-score', 'support'],
      dtype='object')

In [440]:
#agg_cols = {col_name:  ['mean', 'std', 'count'] for col_name, _ in metric_cols.items()}

agg_cols = {col_name:  ['mean', 'std',] for col_name in ['precision', 'recall', 'f1-score']}

overall_df = df[df['class'].isin(['micro avg', 'macro avg'])]\
    .groupby(['system','class','experiment', ])\
    .agg(agg_cols)\
    .unstack().unstack()\
    .swaplevel(0,2,axis=1).swaplevel(1,3,axis=1).sort_index(axis=1)
    

with pd.option_context('display.float_format', lambda x: ('%.3f' % x)[1:]):   
    display(
        overall_df.style.apply(highlight_max)
    )

In [370]:
# pd.set_option('display.float_format', lambda x: '%.3f' % x)

with pd.option_context('display.float_format', lambda x: ('%.3f' % x)[1:]):   
    overall_tex = overall_df.to_latex().split(r'\midrule', 2)
    body_tex, footer_tex = overall_tex[1].split(r'\bottomrule', 2)    
    header_tex = overall_tex[0]
    
    
    # {} &       mean
    
    header_tex = re.sub(r'(.*)mean(.*)', '', header_tex)
    header_tex = re.sub(r'(.*)system(.*)', '', header_tex)
        
    #print(re.search(r'(.*)mean(.*)', header_tex))
    
    print(
        header_tex\
            .replace(r'multicolumn{2}{l}', r'multicolumn{1}{l}')\
            .replace(r'multicolumn{12}{l}', r'multicolumn{6}{l}')\
            .replace(r'multicolumn{6}{l}', r'multicolumn{3}{l}')\
            .replace('lrrrrrrrrrrrr', 'l|r|r|r|r|r|r')
    )
    print(r'\midrule')
    
    for i, line in enumerate(body_tex.split(r' \\')):
        if (i % 2) == 0:
            print(r'\rowcolor{Gray} ')
        
        print(
            line.replace('& .', r' $\pm$.')
        )
        print(r' \\')
        
    print(r'\bottomrule')
    print(footer_tex)

\begin{tabular}{l|r|r|r|r|r|r}
\toprule
experiment & \multicolumn{3}{l}{ACL Anthology} \\
class & \multicolumn{3}{l}{macro avg} & \multicolumn{3}{l}{micro avg} \\
{} & \multicolumn{1}{l}{f1-score} & \multicolumn{1}{l}{precision} & \multicolumn{1}{l}{recall} & \multicolumn{1}{l}{f1-score} & \multicolumn{1}{l}{precision} & \multicolumn{1}{l}{recall} \\



\midrule
\rowcolor{Gray} 

baseline-rnn                   &          .062  $\pm$.001 &      .064  $\pm$.002 &   .059  $\pm$.001 &      .318  $\pm$.004 &      .772  $\pm$.020 &   .201  $\pm$.003
 \\

baseline-rnn\_\_fasttext\_\_custom &          .059  $\pm$.001 &      .100  $\pm$.012 &   .055  $\pm$.002 &      .297  $\pm$.008 &      .766  $\pm$.013 &   .184  $\pm$.006
 \\
\rowcolor{Gray} 

bert-base-cased                &          .258  $\pm$.007 &      .435  $\pm$.087 &   .236  $\pm$.005 &      .648  $\pm$.003 &      .718  $\pm$.002 &   .590  $\pm$.004
 \\

covid\_bert\_base                &          .247  $\pm$.007 &      .368  $\pm$.0

In [371]:
odf.columns.levels

FrozenList([['acl_docrel'], ['macro avg', 'micro avg'], ['f1-score', 'precision', 'recall'], ['mean', 'std']])

# By label class

- label X: sys_1_prec, sys_1_rec, sys_1_f1, sys_2_prec, ...

In [443]:
selected_system = 'scibert-scivocab-uncased'
selected_systems = [
    'bert-base-cased',
    'scibert-scivocab-uncased'
]

label_df = df[df['system'].isin(selected_systems)]\
    .groupby(['class', 'experiment', 'system' ])\
    .agg({c: 'mean' for c in ['f1-score', 'precision', 'recall']})\
    .unstack()\
    .unstack()\
    .swaplevel(0,2,axis=1).sort_index(axis=1)\
    .reindex(sorted_label_classes)

    #.drop(columns=[('ACL Anthology', 'fold'), ('ACL Anthology', 'support')])\
    #.reindex(sorted_label_classes)
    
with pd.option_context('display.float_format', lambda x: ('%.3f' % x)[1:]):  
    display(label_df)

experiment      ACL Anthology                                            \
system        bert-base-cased                  scibert-scivocab-uncased   
                     f1-score precision recall                 f1-score   
class                                                                     
background               .097      .524   .054                     .246   
conclusion               .002      .250   .001                     .000   
discussion               .002      .250   .001                     .000   
evaluation               .000      .000   .000                     .004   
experiment               .314      .517   .227                     .366   
introduction             .488      .551   .438                     .521   
method                   .000      .000   .000                     .000   
previous work            .051      .938   .027                     .294   
related work             .598      .648   .555                     .641   
results                  .000      .000   .000                     .012   
other                    .609      .638   .582                     .629   
none                     .931      .911   .951                     .942   
macro avg                .258      .435   .236                     .305   
micro avg                .648      .718   .590                     .669   
samples avg              .639      .665   .635                     .669   
weighted avg             .611      .664   .590                     .640   

experiment                      
system                          
              precision recall  
class                           
background         .636   .156  
conclusion         .000   .000  
discussion         .000   .000  
evaluation         .250   .002  
experiment         .495   .292  
introduction       .569   .480  
method             .000   .000  
previous work      .718   .186  
related work       .657   .625  
results            .283   .006  
other              .645   .615  
none               .936   .949  
macro avg          .432   .276  
micro avg          .725   .622  
samples avg        .694   .667  
weighted avg       .676   .622

In [372]:
label_classes = set([c.split('__')[0] for c in df.columns if '__' in c and ' avg' not in c])
label_classes

set()

In [189]:
sorted_label_classes = [
    'background',
    'conclusion',
    'discussion',
    'evaluation',
    'experiment',
    'introduction',    
    'method',
    'previous work',
    'related work',
    'results',
    'other',
    'none',
    'macro avg',
    'micro avg',
    'samples avg',
    'weighted avg'
]

#systems = df['system'].unique().tolist()
#footer_classes = ['other', 'none', 'macro avg', 'micro avg', ]


by_label_class_rows = []

for idx, row in df.iterrows():
    for col in row.keys():
        v = col.split('__')
        
        if len(v) == 2 and row['system'] in selected_systems:
            by_label_class_rows.append({
                'label': v[0],
                row['system'] + '__' + v[1]: row[col],
                #'fold': row['fold'],
            })

by_label_class_df = pd.DataFrame(by_label_class_rows)

In [190]:
by_label_class_df.groupby('label').agg({selected_systems[0] + '__support': 'mean'}).values[:,0]

array([  341.25,   289.5 ,   283.  ,   242.75,  1006.25,  4069.75,
       21520.75,   179.75, 21520.75,  6068.75,  5394.25,   200.25,
        3150.  ,   295.25, 21520.75, 21520.75])

In [191]:
by_label_class_df.groupby('label').agg(['mean', 'std', 'count']).reindex(sorted_label_classes)

bert-base-cased__precision                  \
                                    mean       std count   
label                                                      
background                      0.524306  0.102925     4   
conclusion                      0.250000  0.500000     4   
discussion                      0.250000  0.500000     4   
evaluation                      0.000000  0.000000     4   
experiment                      0.516850  0.040845     4   
introduction                    0.550998  0.005105     4   
method                          0.000000  0.000000     4   
previous work                   0.937500  0.125000     4   
related work                    0.647622  0.013010     4   
results                         0.000000  0.000000     4   
other                           0.637806  0.007519     4   
none                            0.910647  0.005430     4   
macro avg                       0.435477  0.087384     4   
micro avg                       0.717877  0.002484     4   
samples avg                     0.665303  0.003255     4   
weighted avg                    0.663731  0.012238     4   

              bert-base-cased__recall                  \
                                 mean       std count   
label                                                   
background                   0.053892  0.032251     4   
conclusion                   0.000842  0.001684     4   
discussion                   0.000845  0.001689     4   
evaluation                   0.000000  0.000000     4   
experiment                   0.227432  0.027428     4   
introduction                 0.437522  0.013988     4   
method                       0.000000  0.000000     4   
previous work                0.027428  0.031565     4   
related work                 0.555243  0.013937     4   
results                      0.000000  0.000000     4   
other                        0.582426  0.009370     4   
none                         0.951349  0.006443     4   
macro avg                    0.236415  0.004551     4   
micro avg                    0.590047  0.004178     4   
samples avg                  0.635215  0.003804     4   
weighted avg                 0.590047  0.004178     4   

              bert-base-cased__f1-score                  \
                                   mean       std count   
label                                                     
background                     0.096560  0.054327     4   
conclusion                     0.001678  0.003356     4   
discussion                     0.001684  0.003367     4   
evaluation                     0.000000  0.000000     4   
experiment                     0.314036  0.019344     4   
introduction                   0.487598  0.007017     4   
method                         0.000000  0.000000     4   
previous work                  0.051268  0.056530     4   
related work                   0.597880  0.013545     4   
results                        0.000000  0.000000     4   
other                          0.608787  0.003984     4   
none                           0.930533  0.003214     4   
macro avg                      0.257502  0.007443     4   
micro avg                      0.647711  0.003220     4   
samples avg                    0.639053  0.003270     4   
weighted avg                   0.611471  0.003922     4   

              bert-base-cased__support  ...  \
                                  mean  ...   
label                                   ...   
background                      341.25  ...   
conclusion                      289.50  ...   
discussion                      283.00  ...   
evaluation                      242.75  ...   
experiment                     1006.25  ...   
introduction                   4069.75  ...   
method                          179.75  ...   
previous work                   200.25  ...   
related work                   3150.00  ...   
results                         295.25  ...   
other                          5394.25  ...   
none     

# Samples

In [192]:
sys_run_results_fp = os.path.join(exp_dir, 'folds', '1', 'bert-base-cased', 'results.csv')

results_cols = ['from_title', 'to_title', 'true', 'predicted']

results_df = pd.read_csv(sys_run_results_fp)
results_df.head()

FileNotFoundError: [Errno 2] File ./output/.ipynb_checkpoints/folds/1/bert-base-cased/results.csv does not exist: './output/.ipynb_checkpoints/folds/1/bert-base-cased/results.csv'

In [ ]:
results_df[results_cols]

In [ ]:
without_predictions = pd.isnull(results_df['predicted']).sum()

print(f'Samples for that no probabilty was above classification threshold: {without_predictions:,} / {len(results_df):,}')

In [ ]:
results_df[results_df['predicted_background'] > 0][results_cols]

In [ ]:
results_df[results_df['true_background'] > 0][results_cols]

In [ ]:
results_df[results_df['predicted_related work'] > 0][results_cols]

In [ ]:
results_df[results_df['true_related work'] > 0][results_cols]

In [ ]:
results_df['true']

In [ ]:
results_df['true'].value_counts()[:20]

In [ ]:
results_df['predicted'].value_counts()

In [ ]:
results_df[results_df['predicted_related work'] > 0][results_cols]

# Multi-label confusion

```

missed classification for... | predicted labels                                | true labels
                                introduction   | background | experiment | ... |
introduction                 |        -        | 12         | 3          | 
(true_introduction=1,predicted_introductioin=0)


```

(color background min/max value)
